In [1]:
import pandas as pd

# Charts for report

A notebook to produce charts for the final report. The charts themselves are made using [Flourish](https://flourish.social/).

Fetch the data needed

In [7]:
charities = pd.read_csv("../data/charities_active.csv", index_col="org_id")
ukcat = pd.read_csv("../data/charities_active-ukcat.csv")

C:\Users\drkan\AppData\Local\Programs\Python\Python38\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (1,8,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [57]:
ukcat_labels = pd.read_csv("../data/ukcat.csv", index_col="Code")
ukcat_labels

,tag,Category,Subcategory,Level,Notes,Related ICNPTSO code,Regular expression,Exclude regular expression
Code,,,,,,,,
AN,Animals,Animals,NaN,1,Includes animals without their own category. I...,E21;E20,\b(animals?|cats?|rabbits?|cows?|sheep|pigs?|c...,NaN
AN101,Cats,Animals,NaN,3,NaN,NaN,\b(cats?|felines?)\b,NaN
AN102,Dogs,Animals,NaN,3,NaN,NaN,\b(dogs?|canines?|kennels?|Rhodesian Ridgeback)\b,NaN
AN103,Donkeys,Animals,NaN,3,NaN,E21;E29;E20,\b(donkey|donkeys|mule(s)?)\b,NaN
AN104,Horses,Animals,NaN,3,NaN,E21;E29;E20,\b(horses?|equine|dressage|(pon(y|ies)+)|horse...,NaN
...,...,...,...,...,...,...,...,...
SO103,Democracy,Society,NaN,3,NaN,G11,\b(democrac(tic|y)|political engagement)\b,NaN
SO104,Equality and diversity,Society,NaN,3,NaN,G11,\b(equality and diversity|(sexual|racial|gende...,NaN
SO105,Human rights,Society,NaN,3,NaN,G11,\b((human|civil|constitutional|civic|natural|u...,NaN


## Number of tags per charity


In [5]:
ukcat.groupby("org_id").size().rename("number_of_tags")

org_id
GB-CHC-1000000     2
GB-CHC-1000001     5
GB-CHC-1000002     2
GB-CHC-1000003     5
GB-CHC-1000005     2
                  ..
GB-SC-SC051140     8
GB-SC-SC051141     5
GB-SC-SC051142    15
GB-SC-SC051143     3
GB-SC-SC051144     6
Length: 194258, dtype: int64

In [12]:
charities = charities.join(ukcat.groupby("org_id").size().rename("number_of_tags"))
charities.loc[:, "number_of_tags"] = charities["number_of_tags"].fillna(0)

In [31]:
n_tags = charities["number_of_tags"].astype(int).value_counts().sort_index()
cutoff = n_tags.index >= 15
over_cutoff = n_tags[cutoff].sum()
n_tags = n_tags[~cutoff]
n_tags.index = n_tags.index.astype(str).rename("Number of tags")
n_tags["15 or over"] = over_cutoff
n_tags = n_tags.rename("Number of charities").to_frame()
n_tags.loc[:, "% of charities"] = (
    n_tags["Number of charities"].divide(n_tags["Number of charities"].sum()).multiply(100).round(1)
)
n_tags

,Number of charities,% of charities
Number of tags,,
0,7705,3.8
1,22409,11.1
2,34082,16.9
3,37411,18.5
4,32587,16.1
5,22906,11.3
6,16743,8.3
7,10350,5.1
8,7522,3.7


## Number of charities with 0 tags, by whether grantmaking is their main activity

In [33]:
charities.loc[charities["number_of_tags"] == 0, "grant_making_is_main_activity"].value_counts()

False    5836
True     1234
Name: grant_making_is_main_activity, dtype: int64

## Number of charities per tag category (direct and indirect)

In [35]:
ukcat.loc[:, "method"] = "direct"

In [65]:
extra = ukcat[ukcat["ukcat_code"].apply(len) > 2]
extra.loc[:, "ukcat_code"] = extra["ukcat_code"].str[0:2]
extra.loc[:, "method"] = "indirect"
all_ukcat = ukcat.append(extra, ignore_index=True).drop_duplicates(["org_id", "ukcat_code"])
all_ukcat

C:\Users\drkan\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,org_id,ukcat_code,method
0,GB-CHC-1000000,ED,direct
1,GB-CHC-1000000,ED102,direct
2,GB-CHC-1000001,AR,direct
3,GB-CHC-1000001,AR104,direct
4,GB-CHC-1000001,AR201,direct
...,...,...,...
1420039,GB-SC-SC051143,SW,indirect
1420042,GB-SC-SC051144,CV,indirect
1420043,GB-SC-SC051144,EC,indirect
1420045,GB-SC-SC051144,LE,indirect


In [68]:
charities_per_tag = (
    pd.crosstab(
        all_ukcat.join(ukcat_labels, on="ukcat_code").loc[all_ukcat["ukcat_code"].apply(len) == 2, "tag"],
        all_ukcat.loc[all_ukcat["ukcat_code"].apply(len) == 2, "method"],
        margins="column",
        margins_name="Total",
    )
    .drop("Total")
    .sort_values("Total", ascending=False)
)
charities_per_tag

method,direct,indirect,Total
tag,,,
Education,56101,28140,84241
Beneficiary group,0,73877,73877
Associations,28809,18911,47720
Religion,17850,26929,44779
Social welfare,0,36623,36623
Charity and VCS support,13581,19652,33233
Arts,11310,18418,29728
Leisure,0,28978,28978
Health,16573,12386,28959


In [70]:
charities_per_tag.divide(len(charities)).multiply(100).round(1)

method,direct,indirect,Total
tag,,,
Education,27.8,13.9,41.7
Beneficiary group,0.0,36.6,36.6
Associations,14.3,9.4,23.6
Religion,8.8,13.3,22.2
Social welfare,0.0,18.1,18.1
Charity and VCS support,6.7,9.7,16.5
Arts,5.6,9.1,14.7
Leisure,0.0,14.3,14.3
Health,8.2,6.1,14.3


## Table used to create appendix A

In [80]:
ukcat_labels.join(ukcat["ukcat_code"].value_counts().rename("Matched charities").astype(int)).reset_index()[
    [
        "Category",
        "Subcategory",
        "tag",
        "Code",
        "Matched charities",
    ]
].rename(columns={"tag": "Tag"}).to_csv("../data/ukcat_for_report.csv", index=False)